# LAB#6 DATASET https://www.kaggle.com/datasets/mitishaagarwal/patient
# Supervivencia de un paciente
# **Github https://github.com/Kel-340/InteligenciaArtificial**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np

import pandas as pd

# Plotting library
from matplotlib import pyplot

# Optimization module in scipy
from scipy import optimize

# will be used to load MATLAB mat datafile format
# from scipy.io import loadmat

# library written for this exercise providing additional functions for assignment submission, and others
# import utils


# tells matplotlib to embed plots within the notebook
%matplotlib inline

In [5]:
# Cargar el dataset
df = pd.read_csv('//content/drive/MyDrive/dataset.csv', sep=",")
#preprocesar los datos cambiar los strings por algun valor numérico
diccionario = {"Caucasian": 0, "African American": 1, "Other/Unknown": 2, "Other/Unknown": 3, "Asian": 4, "Hispanic": 5, "Native American": 6}
df["ethnicity"] = df["ethnicity"].replace(diccionario)

diccionario1 = {"F": 0, "M": 1}
df["gender"] = df["gender"].replace(diccionario1)

diccionario2 = {"Accident & Emergency": 0, "Operating Room / Recovery": 1, "Floor": 2, "Other Hospital": 3, "Other ICU": 4}
df["icu_admit_source"] = df["icu_admit_source"].replace(diccionario2)

diccionario3 = {"admit": 0, "readmit": 1, "transfer": 2}
df["icu_stay_type"] = df["icu_stay_type"].replace(diccionario3)

diccionario4 = {"Med-Surg ICU": 0, "MICU": 1, "Neuro ICU": 2, "CCU-CTICU": 3, "SICU": 4}
df["icu_type"] = df["icu_type"].replace(diccionario4)

diccionario5 = {"Cardiovascular": 0, "Neurological": 1, "Sepsis": 2, "Respiratory": 3, "Gastrointestinal": 4, "Metabolic": 5, "Trauma":6, "Genitourinary":7, "Hematological":8, "Musculoskeletal/Skin":9}
df["apache_3j_bodysystem"] = df["apache_3j_bodysystem"].replace(diccionario5)

diccionario6 = {"Cardiovascular": 0, "Neurologic": 1, "Sepsis": 2, "Respiratory": 3, "Gastrointestinal": 4, "Metabolic": 5, "Trauma":6, "Renal/Genitourinary":7, "Undefined diagnoses":8, "Haematologic":9, "Undefined Diagnoses":10}
df["apache_2_bodysystem"] = df["apache_2_bodysystem"].replace(diccionario6)


# eliminanos la columna vacía
df = df.drop("Unnamed: 83", axis=1)

# Elimina duplicados
df.drop_duplicates(inplace=True)

# Elimina filas con 2 o más valores nulos
df.dropna(axis=0, thresh=2, inplace=True)

# Reemplaza los valores vacíos con 0
df.fillna(0, inplace=True)

# Seleccionar las características (features) y la variable objetivo (target)
X_train = df.iloc[:73000, df.columns != "hospital_death"]
y_train = df.iloc[:73000, df.columns.get_loc("hospital_death")]

X_test = df.iloc[73000:, df.columns != "hospital_death"]
y_test = df.iloc[73000:, df.columns.get_loc("hospital_death")]

X = X_train
y = y_train

y = np.array([int(e) for e in y])
y = np.squeeze(y)

print(X.shape)
print(y.shape)
print(X_test.shape)
print(y_test.shape)

print(X)
print("------------------------------------------------------------")


# Reemplazar los strings restantes con 13
X = X.applymap(lambda x: 13 if isinstance(x, str) else x)

# Verificar que no haya más strings en X
rows_with_strings = X.applymap(lambda x: isinstance(x, str)).any(axis=1)



# Mostrar las filas de X que aún contienen strings (debería estar vacío)
print(X[rows_with_strings])



(73000, 83)
(73000,)
(18713, 83)
(18713,)
       encounter_id  patient_id  hospital_id   age        bmi  \
0             66154       25312          118  68.0  22.730000   
1            114252       59342           81  77.0  27.420000   
2            119783       50777          118  25.0  31.950000   
3             79267       46918          118  81.0  22.640000   
4             92056       34377           33  19.0   0.000000   
...             ...         ...          ...   ...        ...   
72995            91       76807          185  50.0  38.222596   
72996          8871      130448          159  62.0  31.110121   
72997         73248      117775           64  88.0  20.796857   
72998         97016      113823           13   0.0  19.357256   
72999         53911       30158           13  74.0  24.993701   

       elective_surgery  ethnicity  gender  height  icu_admit_source  ...  \
0                     0        0.0     1.0   180.3               2.0  ...   
1                     0

In [ ]:
# Configurando parametros necesario
input_layer_size  = 83  # Entrada de 83 caracteristicas
hidden_layer_size = 10   # 10 unidades ocultas
num_labels = 2         # 2 etiquetas, de 0 a 1

# carga los pesos en las variables Theta1 y Theta2

pesos = {}
pesos['Theta1'] = np.random.rand(10, 84)
pesos['Theta2'] = np.random.rand(2, 11)
# print(pesos['Theta1'][:].shape)
# print(pesos['Theta2'][:].shape)

# print(weights['Theta1'][:].shape)
# print(weights['Theta2'][:].shape)

# print(weights['Theta1'][0])
# print(np.roll(weights['Theta1'][0], 1, axis=0))
# Theta1 tiene un tamaño de 25x401
# Theta2 tiene un tamañó de 10x26
# Theta1, Theta2 = weights['Theta1'], weights['Theta2']
Theta1, Theta2 = pesos['Theta1'], pesos['Theta2']
# se intercambia la ultima columa con la primera de Theta2, por cuestiones de indices que utiliza MATLAB
# print(Theta2)
# print(np.roll(Theta2, 1, axis=0))

# Theta2 = np.roll(Theta2, 1, axis=0)

# Desenrollar parámetros
print(Theta1.ravel().shape)
print(Theta2.ravel().shape)

nn_params = np.concatenate([Theta1.ravel(), Theta2.ravel()])
print(nn_params.shape)

(840,)
(22,)
(862,)


In [ ]:
def sigmoid(z):
    """
    Computes the sigmoid of z.
    """
    return 1.0 / (1.0 + np.exp(-z))

def sigmoidGradient(z):
    """
    Computes the gradient of the sigmoid function for z.
    """
    g = np.zeros(z.shape)
    g = sigmoid(z) * (1 - sigmoid(z))
    return g

def normalize_features(X):
    """
    Normalizes the features of input data X.
    """
    return (X - np.mean(X, axis=0)) / np.std(X, axis=0)

In [ ]:
def nnCostFunction(nn_params,
                   input_layer_size,
                   hidden_layer_size,
                   num_labels,
                   X, y, lambda_=0.0):

    # Reshape nn_params back into the parameters Theta1 and Theta2, the weight matrices
    # for our 2 layer neural network
    Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                        (hidden_layer_size, (input_layer_size + 1)))

    Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                        (num_labels, (hidden_layer_size + 1)))

    m = y.size

    J = 0
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)

    a1 = np.concatenate([np.ones((m, 1)), X], axis=1)

    a2 = sigmoid(a1.dot(Theta1.T))
    a2 = np.concatenate([np.ones((a2.shape[0], 1)), a2], axis=1)

    a3 = sigmoid(a2.dot(Theta2.T))

    # print("-"*20)
    # print(y.shape)
    # print(y.reshape(-1))
    # print("-"*20)
    y_matrix = y.reshape(-1)
    # print(y.shape)
    y_matrix = np.eye(num_labels)[y_matrix]
    # print(y_matrix)

    temp1 = Theta1
    temp2 = Theta2

    # Agregar el termino de regularización

    reg_term = (lambda_ / (2 * m)) * (np.sum(np.square(temp1[:, 1:])) + np.sum(np.square(temp2[:, 1:])))

    J = (-1 / m) * np.sum((np.log(a3) * y_matrix) + np.log(1 - a3) * (1 - y_matrix)) + reg_term

    # Backpropogation

    delta_3 = a3 - y_matrix
    delta_2 = delta_3.dot(Theta2)[:, 1:] * sigmoidGradient(a1.dot(Theta1.T))

    Delta1 = delta_2.T.dot(a1)
    Delta2 = delta_3.T.dot(a2)

    # Agregar regularización al gradiente

    Theta1_grad = (1 / m) * Delta1
    Theta1_grad[:, 1:] = Theta1_grad[:, 1:] + (lambda_ / m) * Theta1[:, 1:]

    Theta2_grad = (1 / m) * Delta2
    Theta2_grad[:, 1:] = Theta2_grad[:, 1:] + (lambda_ / m) * Theta2[:, 1:]

    # ===================== Alterntate solutions =====================
    # my_final_matrix = np.zeros(a3.shape)
    # for c in np.arange(num_labels):
    #    my_final_matrix[:, c] = (np.log(a3[:, c]) * (y == c)) + (np.log(1 - a3[:, c]) * (1 - (y == c)))
    #J = (-1 / m) * np.sum(my_final_matrix)
    # ================================================================

    # ================================================================
    # Unroll gradients
    # grad = np.concatenate([Theta1_grad.ravel(order=order), Theta2_grad.ravel(order=order)])

    grad = np.concatenate([Theta1_grad.ravel(), Theta2_grad.ravel()])

    return J, grad

In [ ]:
lambda_ = 0
J, _ = nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_)
print('Costo en parametros (cargado de ex4weights): %.6f ' % J)

Costo en parametros (cargado de ex4weights): 5.314798 


In [ ]:
z = np.array([-1, -0.5, 0, 0.5, 1])
g = sigmoidGradient(z)
print('Gradiente sigmoide evaluada con [-1 -0.5 0 0.5 1]:\n  ')
print(g)

Gradiente sigmoide evaluada con [-1 -0.5 0 0.5 1]:
  
[0.19661193 0.23500371 0.25       0.23500371 0.19661193]


In [ ]:
def randInitializeWeights(L_in, L_out, epsilon_init=0.12):
    """
    Randomly initialize the weights of a layer in a neural network.

    Parameters
    ----------
    L_in : int
        Number of incomming connections.

    L_out : int
        Number of outgoing connections.

    epsilon_init : float, optional
        Range of values which the weight can take from a uniform
        distribution.

    Returns
    -------
    W : array_like
        The weight initialiatized to random values.  Note that W should
        be set to a matrix of size(L_out, 1 + L_in) as
        the first column of W handles the "bias" terms."""


    W = np.zeros((L_out, 1 + L_in))
    W = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init

    return W

In [ ]:
print('Inicialización de parámetros de redes neuronales...')

initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels)

# Desenrrollr parametros
initial_nn_params = np.concatenate([initial_Theta1.ravel(), initial_Theta2.ravel()], axis=0)

Inicialización de parámetros de redes neuronales...


In [ ]:
options= {'maxiter': 1000}
# Define lambda (parámetro de regularización)
lambda_ = 1

# Crea una función de costo abreviada para minimizar
costFunction = lambda p: nnCostFunction(p, input_layer_size,
                                        hidden_layer_size,
                                        num_labels, X, y, lambda_)

# Ejecuta la optimización
res = optimize.minimize(costFunction,
                        initial_nn_params,
                        jac=True,
                        method='TNC')

# Obtiene la solución de la optimización
nn_params = res.x

# Obtiene Theta1 y Theta2 de nn_params
Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                    (hidden_layer_size, (input_layer_size + 1)))

Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                    (num_labels, (hidden_layer_size + 1)))


<ipython-input-50-fb19a3c25f59>:5: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))


In [ ]:
def predict(Theta1, Theta2, X):
    """
    Predict the label of an input given a trained neural network
    Outputs the predicted label of X given the trained weights of a neural
    network(Theta1, Theta2)
    """
    # Useful values
    m = X.shape[0]
    num_labels = Theta2.shape[0]

    # You need to return the following variables correctly
    p = np.zeros(m)
    h1 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), X], axis=1), Theta1.T))
    h2 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), h1], axis=1), Theta2.T))
    p = np.argmax(h2, axis=1)
    return p

In [ ]:
#valores con los datos de entrenamiento
pred = predict(Theta1, Theta2, X[:100])
accuracy = np.mean(pred == y[:100]) * 100
print(pred)
print('Training Set Accuracy: %.2f%%' % accuracy)

[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0]
Training Set Accuracy: 92.00%


<ipython-input-50-fb19a3c25f59>:5: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))


In [ ]:
#con los datos de prueba

# Reemplazar los strings restantes con 13 en X_test
X_test = X_test.applymap(lambda x: 13 if isinstance(x, str) else x)

# Verificar que no haya más strings en X_test
rows_with_strings_test = X_test.applymap(lambda x: isinstance(x, str)).any(axis=1)

pred = predict(Theta1, Theta2, X_test[:100])
accuracy = np.mean(pred == y_test[:100]) * 100
print(pred)
print('Training Set Accuracy: %.2f%%' % accuracy)

[0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Training Set Accuracy: 90.00%


<ipython-input-50-fb19a3c25f59>:5: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))
